Lab | Cleaning categorical data
For this lab, we will be using the dataset in the Customer Analysis Business Case. This dataset can be found in files_for_lab folder. In this lab we will explore categorical data.

Instructions
Import the necessary libraries if you are starting a new notebook. Using the same data as the previous lab: we_fn_use_c_marketing_customer_value_analysis.csv

In [147]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [148]:
data = pd.read_csv('files_for_lab/we_fn_use_c_marketing_customer_value_analysis.csv')

In [149]:
data.columns = list(map(lambda x: x.lower().replace(' ','_'), data.columns))

Find all of the categorical data. Save it in a categorical_df variable.

In [150]:
categorical_df = data.select_dtypes(include =[object])
categorical_df = categorical_df.drop(['customer'], axis=1)
categorical_df.head()

,state,response,coverage,education,effective_to_date,employmentstatus,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
0,Washington,No,Basic,Bachelor,2/24/11,Employed,F,Suburban,Married,Corporate Auto,Corporate L3,Offer1,Agent,Two-Door Car,Medsize
1,Arizona,No,Extended,Bachelor,1/31/11,Unemployed,F,Suburban,Single,Personal Auto,Personal L3,Offer3,Agent,Four-Door Car,Medsize
2,Nevada,No,Premium,Bachelor,2/19/11,Employed,F,Suburban,Married,Personal Auto,Personal L3,Offer1,Agent,Two-Door Car,Medsize
3,California,No,Basic,Bachelor,1/20/11,Unemployed,M,Suburban,Married,Corporate Auto,Corporate L2,Offer1,Call Center,SUV,Medsize
4,Washington,No,Basic,Bachelor,2/3/11,Employed,M,Rural,Single,Personal Auto,Personal L1,Offer1,Agent,Four-Door Car,Medsize


Check for NaN values.

In [151]:
categorical_df.isna().sum()

state                0
response             0
coverage             0
education            0
effective_to_date    0
employmentstatus     0
gender               0
location_code        0
marital_status       0
policy_type          0
policy               0
renew_offer_type     0
sales_channel        0
vehicle_class        0
vehicle_size         0
dtype: int64

Check all unique values of columns.

In [152]:
categorical_df.apply(pd.Series.unique)

state                [Washington, Arizona, Nevada, California, Oregon]
response                                                     [No, Yes]
coverage                                    [Basic, Extended, Premium]
education            [Bachelor, College, Master, High School or Bel...
effective_to_date    [2/24/11, 1/31/11, 2/19/11, 1/20/11, 2/3/11, 1...
employmentstatus     [Employed, Unemployed, Medical Leave, Disabled...
gender                                                          [F, M]
location_code                                 [Suburban, Rural, Urban]
marital_status                             [Married, Single, Divorced]
policy_type              [Corporate Auto, Personal Auto, Special Auto]
policy               [Corporate L3, Personal L3, Corporate L2, Pers...
renew_offer_type                      [Offer1, Offer3, Offer2, Offer4]
sales_channel                        [Agent, Call Center, Web, Branch]
vehicle_class        [Two-Door Car, Four-Door Car, SUV, Luxury SUV,...
vehicl

In [153]:
display(categorical_df['state'].value_counts(dropna=False))
display(categorical_df['employmentstatus'].value_counts(dropna=False))
display(categorical_df['vehicle_class'].value_counts(dropna=False))
display(categorical_df['vehicle_size'].value_counts(dropna=False)) 

California    3150
Oregon        2601
Arizona       1703
Nevada         882
Washington     798
Name: state, dtype: int64

Employed         5698
Unemployed       2317
Medical Leave     432
Disabled          405
Retired           282
Name: employmentstatus, dtype: int64

Four-Door Car    4621
Two-Door Car     1886
SUV              1796
Sports Car        484
Luxury SUV        184
Luxury Car        163
Name: vehicle_class, dtype: int64

Medsize    6424
Small      1764
Large       946
Name: vehicle_size, dtype: int64

Check dtypes. Do they all make sense as categorical data?

In [154]:
categorical_df.dtypes

state                object
response             object
coverage             object
education            object
effective_to_date    object
employmentstatus     object
gender               object
location_code        object
marital_status       object
policy_type          object
policy               object
renew_offer_type     object
sales_channel        object
vehicle_class        object
vehicle_size         object
dtype: object

In [155]:
categorical_df['effective_to_date'] = pd.to_datetime(categorical_df['effective_to_date'], errors='coerce')

In [167]:
categorical_df.dtypes

state                        object
response                     object
coverage                     object
education                    object
effective_to_date    datetime64[ns]
employmentstatus             object
gender                       object
location_code                object
marital_status               object
policy_type                  object
renew_offer_type             object
sales_channel                object
vehicle_class                object
vehicle_size                 object
dtype: object

Does any column contain alpha and numeric data? Decide how to clean it.
#Renew Offer Type could be just  numeric in this case.	

In [169]:
display(categorical_df['sales_channel'].value_counts(dropna=False)) #Could simply be considered as so for encoding later.

Agent          3477
Branch         2567
Call Center    1765
Web            1325
Name: sales_channel, dtype: int64

In [170]:
display(categorical_df['renew_offer_type'].value_counts(dropna=False))

Offer1    3752
Offer2    2926
Offer3    1432
Offer4    1024
Name: renew_offer_type, dtype: int64

Would you choose to do anything else to clean or wrangle the categorical data? Comment your decisions.

In [172]:
display(categorical_df['employmentstatus'].value_counts(dropna=False))

Employed      6130
Unemployed    3004
Name: employmentstatus, dtype: int64

In [173]:
def cleanOperation(x):
    
    if  (x == "Medical Leave"):
        return "Employed"
    elif ((x == "Disabled") | (x == "Retired")):
        return "Unemployed"
    else:
        return x

# Those who are retired are probably no longer working, although they may choose to do so, but earn a retirement.  
# Disability shouldn't, of course, be a synonym of unemployment, but this answer instead of "Employed" may be an indication.
# Those in Medical Leave are, indeed, employed, or wouldn't fit this label.

In [174]:
display(categorical_df['education'].value_counts(dropna=False))

Undergraduate           5429
High School or Below    2622
Postgraduate            1083
Name: education, dtype: int64

In [175]:
categorical_df['education'] = np.where(categorical_df['education'].isin(['Master','Doctor']) , 'Postgraduate', categorical_df['education'])
categorical_df['education'] = np.where(categorical_df['education'].isin(['College','Bachelor']) , 'Undergraduate', categorical_df['education'])

Compare policy_type and policy. What information is contained in these columns. Can you identify what is important?

In [176]:
categorical_df['employmentstatus'] = list(map(cleanOperation, categorical_df['employmentstatus']))

In [177]:
categorical_df.head()

,state,response,coverage,education,effective_to_date,employmentstatus,gender,location_code,marital_status,policy_type,renew_offer_type,sales_channel,vehicle_class,vehicle_size
0,Washington,No,Basic,Undergraduate,2011-02-24,Employed,F,Suburban,Married,Corporate Auto,Offer1,Agent,Two-Door Car,Medsize
1,Arizona,No,Extended,Undergraduate,2011-01-31,Unemployed,F,Suburban,Single,Personal Auto,Offer3,Agent,Four-Door Car,Medsize
2,Nevada,No,Premium,Undergraduate,2011-02-19,Employed,F,Suburban,Married,Personal Auto,Offer1,Agent,Two-Door Car,Medsize
3,California,No,Basic,Undergraduate,2011-01-20,Unemployed,M,Suburban,Married,Corporate Auto,Offer1,Call Center,Top of Line Cars,Medsize
4,Washington,No,Basic,Undergraduate,2011-02-03,Employed,M,Rural,Single,Personal Auto,Offer1,Agent,Four-Door Car,Medsize


In [182]:
display(categorical_df['policy'].value_counts(dropna=False)) 
display(categorical_df['policy_type'].value_counts(dropna=False))
# Just a name for policy_type. L1, L2 and L3 for each category sum up to the different categories in policy_type. 
#Can be dropped, for policy doesn't add any new info, it's best to have it categorized in three types.
categorical_df = categorical_df.drop(['policy'], axis=1)

KeyError: 'policy'

Check number of unique values in each column, can they be combined in any way to ease encoding? Comment your thoughts and make those changes.

In [179]:

categorical_df['vehicle_class'] = np.where(categorical_df['vehicle_class'].isin(['Sports Car', 'SUV', 'Luxury SUV', 'Luxury Car']) , 'Top of Line Cars', categorical_df['vehicle_class'])
#SUV and luxury are also sports vehicles, besides these are top of line cars, so they could go with luxury cars in general.

In [180]:
display(categorical_df['vehicle_class'].value_counts(dropna=False))

Four-Door Car       4621
Top of Line Cars    2627
Two-Door Car        1886
Name: vehicle_class, dtype: int64

In [181]:
categorical_df.nunique()

state                 5
response              2
coverage              3
education             3
effective_to_date    59
employmentstatus      2
gender                2
location_code         3
marital_status        3
policy_type           3
renew_offer_type      4
sales_channel         4
vehicle_class         3
vehicle_size          3
dtype: int64